# 여러개의 파일을 DataFrame으로 로딩

In [3]:
import pandas as pd
import glob, os
import warnings
warnings.filterwarnings('ignore')

# r''은 Raw 문자열(raw string)을 나타냅니다. Raw 문자열은 이스케이프 문자(escape character)를 해석하지 않고 그대로 문자열로 처리하는 특수한 문자열 형식이다.
path = r'/Users/seojeongsik/Downloads/OpinosisDataset1.0/topics'
#path로 지정한 디렉터리 밑에 있는 모든 .data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path,'*.data'))
filename_list = []
opinion_text = []

# 개별 파일들의 파일명은 filename_list 리스트로 취합,
# 개별 파일들의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환하여 opinion_text 리스트로 취합
for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    
    # 절대경로로 주어진 파일명을 가공.
    filename_ = file_.split('/')[-1]
    filename = filename_.split('.')[0]
    
    # 파일명 리스트와 파일 내용 리스트에 파일명과 파일 내용을 추가.
    filename_list.append(filename)
    opinion_text.append(df.to_string()) # 표형식의 문자열
    
# 파일명 리스트와 파일 내용 리스트를 DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text}) 
document_df.head()

,filename,opinion_text
0,battery-life_ipod_nano_8gb,...
1,gas_mileage_toyota_camry_2007,...
2,room_holiday_inn_london,...
3,location_holiday_inn_london,...
4,staff_bestwestern_hotel_sfo,...


# TfidfVectorizer의 tokenizer인자로 사용될 lemmatization 어근 변환 함수를 설정.

In [5]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

# 입력으로 들어온 token단어들에 대해서 lemmatization 어근 변환. 
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# TfidfVectorizer 객체 생성 시 tokenizer인자로 해당 함수를 설정하여 lemmatization 적용
# 입력으로 문장을 받아서 stop words 제거-> 소문자 변환 -> 단어 토큰화 -> lemmatization 어근 변환. 
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [6]:
# 문서를 TF-IDF 형태로 피처 벡터화
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                             ngram_range=(1,2), min_df=0.05, max_df=0.85)

#opinion_text 칼럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

# 문서 군집화

In [7]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_ # 군집 label 반환
cluster_centers = km_cluster.cluster_centers_ # 군집 좌표 반환

In [8]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,3
1,gas_mileage_toyota_camry_2007,...,2
2,room_holiday_inn_london,...,0
3,location_holiday_inn_london,...,0
4,staff_bestwestern_hotel_sfo,...,0


In [9]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
31,bathroom_bestwestern_hotel_sfo,...,0
17,food_holiday_inn_london,...,0
32,food_swissotel_chicago,...,0
49,free_bestwestern_hotel_sfo,...,0
39,location_bestwestern_hotel_sfo,...,0
3,location_holiday_inn_london,...,0
50,parking_bestwestern_hotel_sfo,...,0
28,price_holiday_inn_london,...,0
2,room_holiday_inn_london,...,0
46,rooms_bestwestern_hotel_sfo,...,0


In [11]:
document_df[document_df['cluster_label' ]== 1].sort_values(by='filename')

,filename,opinion_text,cluster_label
33,accuracy_garmin_nuvi_255W_gps,...,1
34,directions_garmin_nuvi_255W_gps,...,1
48,display_garmin_nuvi_255W_gps,...,1
21,features_windows7,...,1
12,keyboard_netbook_1005ha,...,1
10,satellite_garmin_nuvi_255W_gps,...,1
8,screen_garmin_nuvi_255W_gps,...,1
25,screen_ipod_nano_8gb,...,1
37,screen_netbook_1005ha,...,1
7,size_asus_netbook_1005ha,...,1


In [12]:
document_df[document_df['cluster_label']== 2].sort_values (by='filename')

,filename,opinion_text,cluster_label
18,comfort_honda_accord_2008,...,2
43,comfort_toyota_camry_2007,...,2
1,gas_mileage_toyota_camry_2007,...,2
45,interior_honda_accord_2008,...,2
22,interior_toyota_camry_2007,...,2
35,mileage_honda_accord_2008,...,2
47,performance_honda_accord_2008,...,2
42,quality_toyota_camry_2007,...,2
29,seats_honda_accord_2008,...,2
23,transmission_toyota_camry_2007,...,2


In [13]:
document_df[document_df['cluster_label' ]== 3].sort_values (by='filename')

,filename,opinion_text,cluster_label
9,battery-life_amazon_kindle,...,3
0,battery-life_ipod_nano_8gb,...,3
11,battery-life_netbook_1005ha,...,3
15,performance_netbook_1005ha,...,3
24,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,3


In [15]:
document_df[document_df[ 'cluster_label' ]== 4].sort_values(by='filename')

,filename,opinion_text,cluster_label
26,buttons_amazon_kindle,...,4
36,eyesight-issues_amazon_kindle,...,4
44,fonts_amazon_kindle,...,4
38,navigation_amazon_kindle,...,4
41,price_amazon_kindle,...,4


In [17]:
from sklearn.cluster import KMeans
# 3개의 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect) 
cluster_label= km_cluster.labels_

# 소속 클러스터를 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬 
document_df['cluster_label'] = cluster_label 
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,0
48,display_garmin_nuvi_255W_gps,...,0
44,fonts_amazon_kindle,...,0
41,price_amazon_kindle,...,0
40,speed_windows7,...,0
38,navigation_amazon_kindle,...,0
37,screen_netbook_1005ha,...,0
36,eyesight-issues_amazon_kindle,...,0
34,directions_garmin_nuvi_255W_gps,...,0
33,accuracy_garmin_nuvi_255W_gps,...,0


# 군집별 핵심 단어 추출

In [18]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 4611)
[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]]


In [23]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num,
                        top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array의 값이 가장 큰 순으로 정렬된 인덱스 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:, ::-1]
    
    # 개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화
        cluster_details[cluster_num] = {} # dict의 value를 dict형태로 초기화하여 단어, 위치, 파일명을 입력하려고 한다.
        cluster_details[cluster_num]['cluster'] = cluster_num 
        
        # cluster_centers_.argsort()[:, ::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함.
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [feature_names[ind] for ind in top_feature_indexes]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함
        top_feature_values = cluster_model.cluster_centers_[cluster_num,
                                                            top_feature_indexes].tolist()# 팬시 인덱싱
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('##### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews 파일명 :', cluster_detail['filenames'][:7])
        print('==================================')

In [24]:
feature_names = tfidf_vect.get_feature_names()
cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,
                                      feature_names=feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

##### Cluster 0
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명 : ['battery-life_ipod_nano_8gb', 'voice_garmin_nuvi_255W_gps', 'speed_garmin_nuvi_255W_gps', 'size_asus_netbook_1005ha', 'screen_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'satellite_garmin_nuvi_255W_gps']
##### Cluster 1
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명 : ['gas_mileage_toyota_camry_2007', 'comfort_honda_accord_2008', 'interior_toyota_camry_2007', 'transmission_toyota_camry_2007', 'seats_honda_accord_2008', 'mileage_honda_accord_2008', 'quality_toyota_camry_2007']
##### Cluster 2
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 : ['room_holiday_inn_london', 'location_holiday_inn_london', 'staff_bestwestern_hotel_sfo', 'service_swissotel_hotel_chi